In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from torchvision import models
from sklearn.preprocessing import LabelEncoder
import os

# 设置随机种子以确保可重复性
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# 数据集类
class NPYDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        try:
            self.annotations = pd.read_csv(csv_file, encoding='utf-8')
        except UnicodeDecodeError:
            self.annotations = pd.read_csv(csv_file, encoding='gbk')
        self.root_dir = root_dir
        self.le = LabelEncoder()
        self.annotations['labels'] = self.annotations['labels'].apply(lambda x: x.strip("[]'"))
        self.annotations['labels'] = self.le.fit_transform(self.annotations['labels'])

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, -1]) + '.npy')
        image = np.load(img_name)
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = self.annotations.iloc[index, 2]
        return image, label

# 模型定义
class DenseNet2D(nn.Module):
    def __init__(self, num_classes):
        super(DenseNet2D, self).__init__()
        # 使用预训练的DenseNet模型
        self.densenet = models.densenet121(pretrained=True)
        self.dropout = nn.Dropout(0.5)  # 添加Dropout层
        
        # DenseNet的分类器部分是一个名为classifier的线性层
        # 我们需要用新的线性层替换它，以匹配我们的类别数目
        self.densenet.classifier = nn.Linear(self.densenet.classifier.in_features, num_classes)
        
        # 由于我们已经在上面的行中将原始的分类器替换掉了，
        # 所以这里不需要再替换fc层为Identity

    def forward(self, x):
        # 通过DenseNet模型
        x = self.densenet(x)
        # Dropout层现在不是必需的，因为我们在最后一层之前已经包含了dropout
        # x = self.dropout(x)
        # 由于我们已经将分类器层替换为自定义的线性层，以下步骤也不再需要
        # x = self.fc(x)
        return x

# 训练和评估参数
num_epochs = 10
batch_size = 16
learning_rate = 0.0005

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 数据加载
train_dataset = NPYDataset(csv_file='/kaggle/input/gaus-d-n/train.csv', root_dir='/kaggle/input/gaus-d-n/normalized_gaus_train_images/normalized_gaus_train_images')
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = NPYDataset(csv_file='/kaggle/input/gaus-d-n/validation.csv', root_dir='/kaggle/input/gaus-d-n/normalized_gaus_validation_images/normalized_gaus_validation_images')
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=False)

test_dataset = NPYDataset(csv_file='/kaggle/input/gaus-d-n/test.csv', root_dir='/kaggle/input/gaus-d-n/normalized_gaus_test_images/normalized_gaus_test_images')
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 模型初始化
num_classes = len(np.unique(train_dataset.annotations['labels']))

# 创建一个权重数组
# 这里假设类别标签已经编码为0, 1
weights = torch.tensor([1.0, 1.0], dtype=torch.float32).to(device)

# 使用加权损失函数
criterion = nn.CrossEntropyLoss(weight=weights)

model = DenseNet2D(num_classes=num_classes).to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-1)  # 增加权重衰减

# 学习率调度器
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# 训练模型的代码...
# 请根据您的具体需求添加训练循环和验证/测试循环

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 104MB/s] 


In [2]:
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# 确保模型在GPU上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 确保模型处于训练模式
model.train()

# 可能需要调整学习率
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)  # 示例学习率

for epoch in range(0, 20):  # 继续训练过程
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).long()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # 每个epoch结束后打印损失
    print(f'Epoch [{epoch+1}/{20}], Loss: {loss.item():.4f}')
    
    # 每个epoch结束后在训练集上评估模型
    model.eval()  # 切换到评估模式
    with torch.no_grad():
        train_preds = []
        train_labels = []
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device).long()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            train_preds.extend(predicted.view(-1).cpu().numpy())
            train_labels.extend(labels.view(-1).cpu().numpy())
        train_accuracy = 100 * np.sum(np.array(train_preds) == np.array(train_labels)) / len(train_labels)
        print(f'Train Accuracy: {train_accuracy:.2f} %')

    # 每个epoch结束后在验证集上评估模型
    with torch.no_grad():
        val_preds = []
        val_labels = []
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device).long()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_preds.extend(predicted.view(-1).cpu().numpy())
            val_labels.extend(labels.view(-1).cpu().numpy())
        val_accuracy = 100 * np.sum(np.array(val_preds) == np.array(val_labels)) / len(val_labels)
        print(f'Validation Accuracy: {val_accuracy:.2f} %')
    
    # 每个epoch结束后在测试集上评估模型
    with torch.no_grad():
        test_preds = []
        test_labels = []
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device).long()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            test_preds.extend(predicted.view(-1).cpu().numpy())
            test_labels.extend(labels.view(-1).cpu().numpy())
    
        test_accuracy = 100 * np.sum(np.array(test_preds) == np.array(test_labels)) / len(test_labels)
        print(f'Test Accuracy: {test_accuracy:.2f} %')

        # 计算精确率、召回率和F1分数
        precision = precision_score(test_labels, test_preds, average='weighted')
        recall = recall_score(test_labels, test_preds, average='weighted')
        f1 = f1_score(test_labels, test_preds, average='weighted')
    
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F1 Score: {f1:.4f}')
    
    model.train()  # 切换回训练模式

Epoch [1/20], Loss: 0.4563
Train Accuracy: 69.43 %
Validation Accuracy: 64.67 %
Test Accuracy: 68.00 %
Precision: 0.7498
Recall: 0.6800
F1 Score: 0.6541
Epoch [2/20], Loss: 0.3066
Train Accuracy: 72.57 %
Validation Accuracy: 64.00 %
Test Accuracy: 66.67 %
Precision: 0.7408
Recall: 0.6667
F1 Score: 0.6367
Epoch [3/20], Loss: 0.3816
Train Accuracy: 93.29 %
Validation Accuracy: 80.00 %
Test Accuracy: 82.00 %
Precision: 0.8200
Recall: 0.8200
F1 Score: 0.8200
Epoch [4/20], Loss: 0.6082
Train Accuracy: 93.57 %
Validation Accuracy: 74.00 %
Test Accuracy: 81.33 %
Precision: 0.8379
Recall: 0.8133
F1 Score: 0.8103
Epoch [5/20], Loss: 0.6895
Train Accuracy: 94.43 %
Validation Accuracy: 71.33 %
Test Accuracy: 78.00 %
Precision: 0.8157
Recall: 0.7800
F1 Score: 0.7742
Epoch [6/20], Loss: 0.0756
Train Accuracy: 98.86 %
Validation Accuracy: 82.00 %
Test Accuracy: 82.00 %
Precision: 0.8200
Recall: 0.8200
F1 Score: 0.8200
Epoch [7/20], Loss: 0.0742
Train Accuracy: 98.00 %
Validation Accuracy: 78.67 %
Te